In [3]:
import tensorflow as tf
import numpy as np
import keras

from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from tensorflow.python.saved_model import builder as saved_model_builder

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

from keras import backend as K
K.set_learning_phase(0)
K._LEARNING_PHASE = tf.constant(0) # test mode

K.set_session(sess)

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = np.reshape(x_train, (60000, 784))
y_train = keras.utils.to_categorical(y_train, 10)

x_test = np.reshape(x_test, (10000, 784))
y_test = keras.utils.to_categorical(y_test, 10)

print(x_test[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  84 185 159 151  60  36   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 222 254 254 254
 254 241 198 198 198 198 198 198 198 198 170  52   

In [6]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
# https://github.com/tensorflow/serving/issues/310#issuecomment-297015251

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [8]:
import sys
old = sys.stdout
sys.stdout = open('keras_output.txt', 'w')
history = model.fit(x_train, y_train,
                    batch_size=1000,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
sys.stdout = old

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 5.2510824645996097)
('Test accuracy:', 0.67349999999999999)


In [10]:
num_output = 10
prefix_output_node_names_of_final_network = 'output_node'

pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

('output nodes names are: ', ['output_node0', 'output_node1', 'output_node2', 'output_node3', 'output_node4', 'output_node5', 'output_node6', 'output_node7', 'output_node8', 'output_node9'])


In [11]:
import os.path as osp

#print(sess.graph.as_graph_def())

sess = K.get_session()
output_fld = './export'
output_graph_name = 'saved_mnist.pb'

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))

INFO:tensorflow:Froze 4 variables.
Converted 4 variables to const ops.
('saved the constant graph (ready for inference) at: ', './export/saved_mnist.pb')
